In [19]:
#compute dim D^{\lambda}
#define action of S_n on Young tableaux - DONE
#for a p-regular partition \lambda
#find basis elements e_i for Specht modules S^\lambda
#write basis e_T in terms of Young tableaux as in Chan's notes
#compute rank of Gram matrix (<e_i,e_j>)_{i,j=1,...,n} as in Chan's notes

In [20]:
#REFS:
#Chan's notes - https://math.mit.edu/~charchan/ModularRepresentationsSymmetricGroupSeminar.pdf

In [21]:
#https://en.wikipedia.org/wiki/Iwahori–Hecke_algebra
#exists more general construction using Hecke algebras associated to Coxeter groups, i.e. Iwahori-Hecke algebras
#when q=1, we get the symmetric group algebra
#there are *two* bases, the canonical one and another discovered by Kazhdan-Lusztig
#one of these bases is equivalent to the basis described in Chan's notes
#the Kazhdan-Lusztig polynomials essentially give the transition matrix from one to the other
#computing the size of this basis over F_p gives the dimension of D^\lambda
#Travis Scrimshaw has implemented this code in https://github.com/sagemath/sage/pull/36718
""""
SGA = SymmetricGroupAlgebra(GF(3), 5)
C = SGA.cellular_basis()
for la in C.cell_poset():
    M = C.cell_module(la)
    if not M.nonzero_bilinear_form():
        continue
    (la, M.dimension(), M.simple_module().dimension())
"""

'"\nSGA = SymmetricGroupAlgebra(GF(3), 5)\nC = SGA.cellular_basis()\nfor la in C.cell_poset():\n    M = C.cell_module(la)\n    if not M.nonzero_bilinear_form():\n        continue\n    (la, M.dimension(), M.simple_module().dimension())\n'

In [22]:
#define action of S_n on Young tableaux T
def act(sigma,T):
    return Tableau([[sigma(l[i]) for i in range(len(l))] for l in T])

In [23]:
#define action on rows of tablueax
#g is an element of the subgroup \prod_i S_{n_i} where \sum_i n_i = n
#T is a tableau corresponding to some partition T.shape()
def act_rows(g,T):
    return [g[i](T[i]) for i in range(len(T))]

In [24]:
#determine when two tableaux are row-equivalent
def row_equiv(T_0,T_1):
    assert T_0.shape() == T_1.shape()
    return all(set(T_0[i])==set(T_1[i]) for i in range(len(T_0)))

In [25]:
#get columns of tableau
#there is a MUCH smarter way to do this
def cols(T):
    cols=[]
    for j in range(max(T.shape())):
        col=[]
        for i in range(len(T)):
            try:
                col.append(T[i][j])
            except:
                IndexError
        cols.append(col)
    return cols

In [26]:
#compute the column stabilizer of a tableau T
def col_stab(T):
    n=sum(T.shape())
    col_1 = cols(T)
    col_stabilizer = []
    for g in SymmetricGroup(n):
        col_2 = cols(act(g,T))
        assert len(col_1) == len(col_2)
        cols_preserved = True
        for i in range(len(col_1)):
            if set(col_1[i]) != set(col_2[i]):
                cols_preserved = False
        if cols_preserved:
            col_stabilizer.append(g)
    return col_stabilizer

In [27]:
#compute polytabloid e_T associated to a standard young tableau T
#e_T = \sum_{\sigma \in C_T} sgn(\sigma) { \sigma T}
#where {} is the row-equivalence class, i.e. a tabloid
#C_T is the column stabilizer, the subgroup of the symmetric group preserving the columns of tableau T setwise
#sgn is the sign of the permutation \sigma
#the sum takes place in the module generated by tabloids {T}
#represent lienar combination of tabloids as a dict={Tableau:Integer}
#if T_1 is equivalent to T_2, just increase the sum by sgn(\sigma)
#if T_1 is not equivalent to T_2, create a new entry
def polytabloid(T):
    e_T = {}
    C_T = col_stab(T)
    for pi in C_T:
        T_1 = act(pi,T)
        sum_contains_equiv = False
        for T_2 in e_T:
            if row_equiv(T_1,T_2):
                sum_contains_equiv = True
                e_T[T_2] += sgn(pi)
        if not sum_contains_equiv:
            e_T[T_1] = sgn(pi)
    return e_T

In [28]:
#compute bilinear form <.,.> associated to span of tabloids
#<{s},{t}> = \delta_{{s},{t}}
#takes in two dicts representing polytabloids e_{T_1} and e_{T_2}
#iterates through both keys only increasing the sum by the product of entries when {s}=={t}, i.e. s ~ t
def bilinear_form(poly_1,poly_2):
    sum = 0
    for s in poly_1:
        for t in poly_2:
            if row_equiv(s,t):
                sum += poly_1[s]*poly_2[t]
    return sum

In [29]:
#create a standard "canonical" tableau by placing entries sequentially from left-to-right, top-to-bottom
def canonical_tableau(la):
    return Tableau([[sum(la[k] for k in range(j))+i+1 for i in range(la[j])] for j in range(len(la))])

In [30]:
#use the bilinear form to compute the rank of the Gram matrix (<e_i,e_j>)_{i,j=1,...,n}
#note that polytabloids e_T form a basis for S^\lambda for *standard* tableau T
def gram_matrix(la):
    return [[bilinear_form(polytabloid(T_1),polytabloid(T_2)) for T_1 in StandardTableaux(la)] for T_2 in StandardTableaux(la)]

In [31]:
#compute the rank of a matrix modulo p when given as a list
def rank(A,p):
    return matrix(GF(p),len(A),A).rank()

In [32]:
n=6 #size of symmetric group
P=Primes()[:5]; P #primes p until all all partitions are p-regular

[2, 3, 5, 7, 11]

In [ ]:
header_row=[["p//partition"]+[la for la in Partitions(n)]]; header_row
data=[[str(p)]+[rank(gram_matrix(la),p) if la.is_regular(p) else None for la in Partitions(n)] for p in P]

In [ ]:
header_row+data

In [ ]:
import pandas as pd
df=pd.DataFrame(data,columns=header_row)
print(df.to_string(index=False))

In [ ]:
df.to_csv("rank_gram_matrices_n=6.csv",index=False)